In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)

In [5]:
# use large scale here to get the output really close to the value in dataset, which is important to Median.
# Be careful, since we may need to calculate average of 2 values in case of no middle value in dataset,
# this means larger scale can still blow up circuit size, unlike Mode func that scale doesnt affect circuit size much.
scales = [6]
selected_columns = ['col_name']
generate_data_commitment(data_path, scales, data_commitment_path)

In [6]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    return s.median(x)

# error here in Median only matters in determining the median value in case it doesnt exist in dataset. (Avg of 2 middle values)
error = 0.01
# Prover/ data owner side
_, prover_model = computation_to_model(computation, precal_witness_path, True, selected_columns, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:254: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 6, param_scale: 6, scale_input_multiplier: 1) ------------->

+--------------+--------------+-------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error   | median_error | max_error   | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+--------------+--------------+-------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0.0015621185 | 0.003124237  | 0.003124237 | 0         | 0.0015621185   | 0.003124237      | 0.003124237   | 0             | 0.0000048804286    | 0.000031526106     | 0.000031526106         |
+--------------+--------------+-------------+-----------+----------------+------------------+---------------

scale:  [6]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":6,"param_scale":6,"scale_rebase_multiplier":1,"lookup_range":[-12400,19200],"logrows":15,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":14432,"total_assignments":20182,"total_const_size":9031,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,6],"model_input_scales":[6],"module_sizes":{"kzg":[],"poseidon":[14432,[1]]},"required_lookups":[{"GreaterThan":{"a":0.0}},"ReLU",{"Floor":{"scale":64.0}}],"required_range_checks":[[-32,32],[-1,1]],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1715310676130}


In [7]:
_, verifier_model = computation_to_model(computation, precal_witness_path, False, selected_columns, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 3.9898521900177 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.546875
==== Generating Proof ====
proof:  {'instances': [['2363efbe326292a2c2e7954acfd2ce00d744f0ebeda83508d86a90913457461a', '0100000000000000000000000000000000000000000000000000000000000000', '630c000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x261f28fe7695d7be0c72aff5404349a08f6f732436e7493e1b2fc91d7d57deb32a1500642bb845cb8fa4be77b22e812fc1c85c0691464be3e3150f3e9a58d78816548cf9272a1b9b5f64940c211a32986558c6151a18c7cf816de5dc0980044e28b14e7bdb1c17c8904b9cedb1b8420d3a7642c8c3d2ebfc7d4281fdcce2cf572dc03a30fbd5674cb11d7a7023dd33bbeb0a167a06c72ced7d0368dca77c02b90961d2ccb81ca09edf7ed2508549a9ee7442a28f441173b1791a54abb2642a65170799f9d1de7fcb01740e1dcd7973e20147d9b8dee366282bcbd82da4954efb1835a07f75c18793a7a168036471ff8a9ea0cd137b7ab3fa1fd0f466640c46e917ef2a76bd60cbe0f0539eea84d17f9f6111163d83e3f1b382080ccb8beb27da04adb1ffe8beb

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [49.546875]
